<a href="https://colab.research.google.com/github/MihaiDogariu/Keysight-Deep-Learning-Fundamentals--v2-/blob/main/scripts/Unit_8_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regression

This notebook addresses the problem of regression: having a set of explanatory variables, we are interested to see if we can connect a dependent variable to them. In other words we want to predict one of the object's attributes based on the values of the rest of its attributes.

This application aims to predict the fuel consumption of a set of cars, based on several other indicators.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import torch
from torch import nn

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

Download and inspect the dataset

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
raw_dataset

Auxiliary function to help with transforming from 'miles per gallon' to 'litres per 100 km'.

In [ ]:
def mpg_to_lper100km(mpg):
  gallon = 3.785411784 # 1 gallon = 3.785411784 litres
  mile = 1.609344 # 1 mile = 1.609344 km
  lper100km = 100 * (gallon / mile) / mpg
  return lper100km

Apply the above transformation on the entire dataset.

In [ ]:
dataset = raw_dataset.rename(columns={"MPG": "Consumption"})
dataset["Consumption"] = dataset["Consumption"].apply(lambda x: mpg_to_lper100km(x))
dataset

Let's see how the data is correlated throughout the dataset.

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(dataset.corr(),annot=True,annot_kws={"size":10})

As it happens most of the time, the dataset might contain NULL or NaN values. We must find out if that also happens now.

In [ ]:
dataset.isna().sum()

We drop the NaN values since they represent only a tiny portion of the dataset. We plot the correlation matrix again, after dropping the features that do not hold a high enough correlation value with our w.

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset = dataset.drop(columns=["Acceleration", "Model Year", "Origin"])
plt.figure(figsize=(10,6))
sns.heatmap(dataset.corr(),annot=True,annot_kws={"size":12})

Let's plot some more info about the dataset.

In [ ]:
dataset.describe()

In [ ]:
plt.hist(dataset['Consumption'],histtype='bar')

We can see that there are quite a few outliers for the 'Horsepower' attribute

In [ ]:
dataset.boxplot(column="Horsepower")

Auxiliary function to detect outliers and help reduce the dataset only to the 25-75 quartile range.

In [ ]:
def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []

    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        # Q1 = np.percentile(df[col], 25)
        Q1 = df[col].quantile(0.25)
        # 3rd quartile (75%)
        # Q3 = np.percentile(df[col],75)
        Q3 = df[col].quantile(0.75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1

        # outlier step
        outlier_step = 1.5 * IQR

        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index

        # append the found outlier indices for col to the list of outlier indices
        outlier_indices.extend(outlier_list_col)

    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )

    return multiple_outliers

In [ ]:
drop = detect_outliers(dataset,0,['Consumption', 'Cylinders','Displacement','Horsepower','Weight'])
dataset = dataset.drop(drop, axis = 0).reset_index(drop=True)

In [ ]:
drop

In [ ]:
dataset.boxplot(column="Horsepower")

In [ ]:
log_dataset = dataset.copy()
log_dataset["Consumption"] = dataset["Consumption"].apply(lambda x: np.log(1 + 100*x))

f, axes = plt.subplots(1, 2)

sns.kdeplot(dataset[["Consumption"]], ax=axes[0], fill=True)
sns.kdeplot(log_dataset[["Consumption"]], ax=axes[1], fill=True)

Since the data seems to be skewed, we can use a transformation that will help straighten it up, such as the log transform.

In [ ]:
use_log_transform = True

if use_log_transform:
  dataset = log_dataset

Now that we are finally done with the data pre-processing, we can move on and train our network. First, we create the train-test subsets.

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
train_features = train_dataset.drop(columns=["Consumption"])
train_labels = train_dataset["Consumption"]

test_dataset = dataset.drop(train_dataset.index)
test_features = test_dataset.drop(columns=["Consumption"])
test_labels = test_dataset["Consumption"]

We define the model architecture using the `torch.nn.Sequential()` approach.

In [ ]:
# Define network dimensions
n_input_dim = train_features.shape[1]
# Layer size
n_hidden = 10 # Number of hidden nodes
n_output = 1 # Number of output nodes for predicted consumption

# Build mdel
torch_model = torch.nn.Sequential(
    torch.nn.Linear(n_input_dim, n_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(n_hidden, n_output),
)

print(torch_model)

Defining the loss, learning rate and optimizer.

In [ ]:
loss_func = torch.nn.MSELoss() #Choosing mean square error as loss metric
learning_rate = 0.001
optimizer = torch.optim.Adam(torch_model.parameters(), lr=learning_rate)

Performing the actual training.

In [ ]:
train_error = []
test_error = []
iters = 2000

y_train_t = torch.FloatTensor(train_labels.values).reshape(-1,1) #Converting numpy array to torch tensor
x_train_t = torch.nn.functional.normalize(torch.FloatTensor(train_features.values), dim=0)

y_test_t = torch.FloatTensor(test_labels.values).reshape(-1,1) #Converting numpy array to torch tensor
x_test_t = torch.nn.functional.normalize(torch.FloatTensor(test_features.values), dim=0)

for i in range(iters):
    torch_model.train(True)
    # x_train_t = torch.FloatTensor(train_features.values)  #Converting numpy array to torch tensor
    y_train_hat = torch_model(x_train_t)
    # print(torch.transpose(y_hat, 0, 1))
    loss = loss_func(y_train_hat, y_train_t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    train_error.append(loss.item())

    torch_model.train(False)
    with torch.no_grad():
      y_test_hat = torch_model(x_test_t)
      test_loss = loss_func(y_test_hat, y_test_t)
      test_error.append(test_loss.item())


x = np.linspace(start=1, stop=iters, num=iters)
fig, ax = plt.subplots(1, 1, figsize=(12,8))
ax.plot(x, train_error, color="red", label="Train Loss")
ax.plot(x, test_error, color="blue", label="Test Loss")
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.set_title('Loss evolution')
ax.legend()